In [1]:
import ecc_tools as tools
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
#import emachine as EM
from direct_info import direct_info
from ecc_tools import distance_restr

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from scipy.sparse import csr_matrix
from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import numpy as np
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import expectation_reflection as ER
from direct_info import direct_info
from joblib import Parallel, delayed

# import pydca for plmDCA
from pydca.plmdca import plmdca
from pydca.meanfield_dca import meanfield_dca
from pydca.sequence_backmapper import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.contact_visualizer import contact_visualizer
from pydca.dca_utilities import dca_utilities

tools.hide_toggle()

## Using comprehensive software package for DCA
#### Author: Mehari B. Zerihun
article: https://academic.oup.com/bioinformatics/advance-article/doi/10.1093/bioinformatics/btz892/5645176
github: https://github.com/KIT-MBS/pydca
* mean field and pseudoliklihood methos
* several utility functions
    * MSA triming functions
    * Contact Map Plotting Functions

In [2]:
# Read in Protein structure
data_path = '../Pfam-A.full'
pfam_id = 'PF00186'
pdb = np.load('%s/%s/pdb_refs.npy'%(data_path,pfam_id))

# Pre-Process Structure Data
# delete 'b' in front of letters (python 2 --> python 3)
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0]) \
         for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])

# Print number of pdb structures in Protein ID folder
npdb = pdb.shape[0]
print('number of pdb structures: ',npdb)

# Print PDB array 
#print(pdb)
#print(pdb[0])

# Create pandas dataframe for protein structure
df = pd.DataFrame(pdb,columns = ['PF','seq','id','uniprot_start','uniprot_start',\
                                 'pdb_id','chain','pdb_start','pdb_end'])
print("pdb ids:\n",df['id'])
print(df.head())

ipdb = 0
print('seq:',int(pdb[ipdb,1]))

from data_processing import data_processing
from data_processing import write_FASTA

# data processing
s0,cols_removed,s_index = data_processing(data_path,pfam_id,ipdb,\
                gap_seqs=0.2,gap_cols=0.2,prob_low=0.004,conserved_cols=0.9)
print(s0.shape)
#print("Removed columns: ",cols_removed)


FASTA_data_outfile, FASTA_ref_outfile = write_FASTA(s0,pfam_id,pdb,ipdb)

#print(np.array(trimmed_data).shape)
#print(len(trimmed_data[0][1]))
#print(len(trimmed_data[5][1]))
tools.hide_toggle()

number of pdb structures:  372
pdb ids:
 0      Q5KZ26_GEOKA
1      Q5KZ26_GEOKA
2      Q81R22_BACAN
3      Q81R22_BACAN
4      Q81R22_BACAN
           ...     
367       DYR_MYCTU
368       DYR_MYCTU
369       DYR_MYCTU
370    Q834R2_ENTFA
371    Q834R2_ENTFA
Name: id, Length: 372, dtype: object
        PF seq            id uniprot_start uniprot_start pdb_id chain  \
0  PF00186  69  Q5KZ26_GEOKA             1           160   1ZDR     B   
1  PF00186  69  Q5KZ26_GEOKA             1           160   1ZDR     A   
2  PF00186  83  Q81R22_BACAN             2           160   3JWK     B   
3  PF00186  83  Q81R22_BACAN             2           160   3S9U     B   
4  PF00186  83  Q81R22_BACAN             2           160   3FL9     H   

  pdb_start pdb_end  
0         1     160  
1         1     160  
2         2     160  
3         2     160  
4         2     160  
seq: 69
(6734, 137)
Reference Sequence number:  69
Reference Sequence:
 ['-' '-' 'L' 'I' 'V' 'Y' 'A' 'A' 'N' 'R' 'V' 'R' 'D' 'N' 'T

We import pydca. We Import and process MSA file and write the processed MSA data to an output file MSA_PF00186.fa in FASTA format. Next we carry out DCA computations using the processed MSA saved in MSA_PF00186.fa

### DCA Computation Using Pseudolikelihood Maximization Algorithm (plmDCA)

In [3]:
# Compute DCA scores using Pseudolikelihood maximization algorithm

plmdca_inst = plmdca.PlmDCA(
    FASTA_data_outfile,
    'protein',
    seqid = 0.8,
    lambda_h = 1.0,
    lambda_J = 20.0,
    num_threads = 10,
    max_iterations = 500,
)
#print("properties of plmDCA object: \n",dir(plmdca_inst),"\n\n")

sorted_DI = plmdca_inst.compute_sorted_DI()

N = plmdca_inst.sequences_len
B = plmdca_inst.num_sequences
print("sorted_DI (list) has shape: ",np.array(sorted_DI).shape,"\nRows: all combinations of sequence postitions ie sequence len choose 2 ")
print("num sequences = %d , sequences len = %d\n\n"%(B,N))

print("Print top 10 pairs")
for site_pair, score in sorted_DI[:10]:
    print(site_pair, score)


# compute DCA scores summarized by Frobenius norm and average product corrected
plmdca_FN_APC = plmdca_inst.compute_sorted_FN_APC()
tools.hide_toggle()

sorted_DI (list) has shape:  (9316, 2) 
Rows: all combinations of sequence postitions ie sequence len choose 2 
num sequences = 1 , sequences len = 137


Print top 10 pairs
(35, 62) 1.860295978989739e-05
(35, 85) 1.860295978989739e-05
(62, 85) 1.8602948379799547e-05
(35, 96) 1.8602948240792008e-05
(35, 107) 1.8602948240792008e-05
(35, 129) 1.8602948240792008e-05
(35, 136) 1.8602948240792008e-05
(35, 47) 1.860294026470782e-05
(35, 59) 1.860294026470782e-05
(35, 68) 1.860294026470782e-05


## PlmDCA for Protein PF00186. 
* we used the data processed sequence alignment (our version) and not the trimmed MSA data from pydca
* We also set the values of optional parameters. 
* pydca can also be used to compute DCA scores from the Frobenius norm of the couplings by calling the compute_sorted_FN_APC() method on plmdca_inst
    * This action returns the average product corrected (APC) DCA scores. Let us print the top five site pairs and their DCA scores.

In [4]:
# Generate Direct Information matrix
DI = np.zeros((N,N))
DI_ver= np.array(sorted_DI)
for coupling in sorted_DI:
    DI[coupling[0][0],coupling[0][1]] = coupling[1]
print("Calculated DI: (shape, matrix)\n",DI.shape,"\n",DI)
np.save("plmDCA_DI_%s.npy"%(pfam_id),DI)

tools.hide_toggle()

Calculated DI: (shape, matrix)
 (137, 137) 
 [[0.00000000e+00 1.86024055e-05 1.86021369e-05 ... 1.86021399e-05
  1.86021399e-05 1.86024800e-05]
 [0.00000000e+00 0.00000000e+00 1.86025300e-05 ... 1.86025302e-05
  1.86025302e-05 1.86028591e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.86022729e-05
  1.86022729e-05 1.86026046e-05]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.86022758e-05 1.86026075e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.86026075e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


## Visualization in pydca
* Need Reference Sequence to contain no gaps
    * This is an option in data_processing (ie find and replace all '-')
    * Have not tested accurace of methods with randomized replace ment of gaps

In [5]:
# compute DCA scores summarized by Frobenius norm and average product corrected
plmdca_FN_APC = plmdca_inst.compute_sorted_FN_APC()

plmdca_visualizer = contact_visualizer.DCAVisualizer('protein', pdb[ipdb,6], pdb[ipdb,5],
    refseq_file = FASTA_ref_outfile,
    sorted_dca_scores = plmdca_FN_APC,
    linear_dist = 4,
    contact_dist = 8.0,
)

contact_map_data = plmdca_visualizer.plot_contact_map()
tp_rate_data = plmdca_visualizer.plot_true_positive_rates()


	The sequence --LIVYAANRVRDNTLPRLPGDLAHKRSLGHPVINWESLGRPPGLNIVISRNSEYEAAGASVYPSLEAALAACTAEASPYIIAQIYRLALPLARRVVAEVHASIEGAYFQLPGDEWMETSRSERQTENGYEYDFVVY
	is neither protein nor RNA. Does it contain non-standared residues?
	Please note that reference sequences are assumed to contain
	only standared residues.


RefSeqContentException: 